# Figure S2: Seasonal performance in selected basin with glaciers

In [ ]:
import os 
import numpy as np
import pandas as pd
import geopandas as gpd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.chdir('/home/rooda/OneDrive/Projects/DeepHydro/')
path_pmet = "/home/rooda/OneDrive/Projects/PatagoniaMet/"

## Data

In [ ]:
# pmet-obs
selection = gpd.read_file("data/GIS/Basins_PMETobs_points_subset.gpkg").set_index("gauge_id")
pmet_metadata = pd.read_csv(path_pmet + "data/Zenodo/v11/Q_PMETobs_v11_metadata.csv", index_col = 0)
pmet_metadata = pmet_metadata.loc[selection.index]

pmet_data = pd.read_csv(path_pmet + "data/Zenodo/v11/Q_PMETobs_1950_2020_v11d.csv")
pmet_data["Date"] = pd.to_datetime(pmet_data["Date"])
pmet_data = pmet_data.set_index("Date")
pmet_data = pmet_data['2000-01-01' :'2019-12-31']
pmet_data = pmet_data[selection.index]
pmet_data = (pmet_data*1000*86400) / (pmet_metadata.total_area*1e6)

In [ ]:
# experiment
exp = "PUB"

# time series
ts_lstm_oggm_on  = pd.read_csv("results/runoff/total_runoff_historical_CV_{}_LSTM_OGGM_on.csv".format(exp), parse_dates= ["date"], index_col = 0)
ts_lstm_oggm_on = ts_lstm_oggm_on.where(~pmet_data.isna())

ts_lstm_oggm_off = pd.read_csv("results/runoff/total_runoff_historical_CV_{}_LSTM_OGGM_off.csv".format(exp), parse_dates= ["date"], index_col = 0)
ts_lstm_oggm_off = ts_lstm_oggm_off.where(~pmet_data.isna())

ts_gr4j = pd.read_csv("results/runoff/total_runoff_historical_GR4J_pmet_CV_{}.csv".format(exp), index_col = 0)
ts_gr4j.index = pd.to_datetime(ts_gr4j.index)
ts_gr4j = ts_gr4j.where(~pmet_data.isna())

ts_tuwmodel = pd.read_csv("results/runoff/total_runoff_historical_TUWmodel_pmet_CV_{}.csv".format(exp), index_col = 0)
ts_tuwmodel.index = pd.to_datetime(ts_tuwmodel.index)
ts_tuwmodel = ts_tuwmodel.where(~pmet_data.isna())

# aggregate
pmet_data_mean  = pmet_data.groupby(pmet_data.index.dayofyear).median().rolling(7).mean()
pmet_data_q25   = pmet_data.groupby(pmet_data.index.dayofyear).quantile(0.25)
pmet_data_q75   = pmet_data.groupby(pmet_data.index.dayofyear).quantile(0.75)

ts_lstm_oggm_on  = ts_lstm_oggm_on.groupby(ts_lstm_oggm_on.index.dayofyear).median().rolling(7).mean()
ts_lstm_oggm_off = ts_lstm_oggm_off.groupby(ts_lstm_oggm_off.index.dayofyear).median().rolling(7).mean()
ts_gr4j          = ts_gr4j.groupby(ts_gr4j.index.dayofyear).median().rolling(7).mean()
ts_tuwmodel      = ts_tuwmodel.groupby(ts_tuwmodel.index.dayofyear).median().rolling(7).mean()

## Plot

In [ ]:
basins_xy  = ['X10514001', # Puelo
              'X11545000', # Baker Bajo Ñadis
              'X11711000', # Pascua
              'X00002802', # Santa_Cruz_Charles_Fuhr
              'X12287001', # Grey
              'X12289001'] # Serrano

basins_xy  = np.reshape(basins_xy, (2, 3))
cl = px.colors.qualitative.D3

fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.04, vertical_spacing= 0.06, shared_xaxes= True)

for x in range(0,2): 
    for y in range(0,3): 
        
        fig.add_trace(go.Scatter(x=pmet_data_q25.index, y=pmet_data_q25[basins_xy[x,y]], mode='lines',
                                        line=dict(width=0), fillcolor= "rgba(0, 0, 0, 0.1)",  showlegend=False), row=x+1, col=y+1)
        fig.add_trace(go.Scatter(x=pmet_data_q75.index, y=pmet_data_q75[basins_xy[x,y]], mode='lines',
                                       line=dict(width=0), fillcolor="rgba(0, 0, 0, 0.1)", fill='tonexty', showlegend=False), row=x+1, col=y+1)

        fig.add_trace(go.Scatter(x= ts_gr4j.index, y= ts_gr4j[basins_xy[x,y]], mode='lines', name="GR4J + OGGM", marker_color= cl[1], 
                                 line=dict(width = 1.2, dash = "dot"), showlegend = False, opacity = 0.8), row = x+1, col = y+1)

        fig.add_trace(go.Scatter(x= ts_tuwmodel.index, y= ts_tuwmodel[basins_xy[x,y]], mode='lines', name="TUWmodel + OGGM", marker_color= cl[4], 
                                 line=dict(width = 1.2, dash = "dot"), showlegend = False, opacity = 0.8), row = x+1, col = y+1)

        fig.add_trace(go.Scatter(x= ts_lstm_oggm_off.index, y= ts_lstm_oggm_off[basins_xy[x,y]], mode='lines', name="Only LSTM", marker_color= cl[2], 
                                 line=dict(width = 1.2, dash = "dot"), showlegend = False, opacity = 0.8), row = x+1, col = y+1)
        
        fig.add_trace(go.Scatter(x= ts_lstm_oggm_on.index,  y= ts_lstm_oggm_on[basins_xy[x,y]], mode='lines', name="LSTM + OGGM", marker_color= cl[0], 
                                 line=dict(width = 1.5), showlegend = False, opacity = 0.8), row = x+1, col = y+1)
        
        fig.add_trace(go.Scatter(x= pmet_data_mean.index,   y= pmet_data_mean[basins_xy[x,y]], mode='lines', name="Observed", marker_color= "Black", 
                                 line=dict(width = 1.5), opacity = 0.7, showlegend = False), row = x+1, col = y+1)

# annotations (names + glacier cover + latitude)
fig.add_annotation(text="a) Puelo ({:.1f}%; {:.1f}°S)".format(pmet_metadata.glacier_cover.loc["X10514001"], -pmet_metadata.gauge_lat.loc["X10514001"]),      
                   font=dict(size=16), xref = "paper", yref = "paper", x=0,  y=0.55, showarrow=False)

fig.add_annotation(text="b) Baker ({:.1f}%; {:.1f}°S)".format(pmet_metadata.glacier_cover.loc["X11545000"], -pmet_metadata.gauge_lat.loc["X11545000"]),      
                   font=dict(size=16), xref = "paper", yref = "paper", x=0.44,  y=0.55, showarrow=False)

fig.add_annotation(text="c) Pascua ({:.1f}%; {:.1f}°S)".format(pmet_metadata.glacier_cover.loc["X11711000"], -pmet_metadata.gauge_lat.loc["X11711000"]),   
                   font=dict(size=16), xref = "paper", yref = "paper", x=0.88,  y=0.55,  showarrow=False)

fig.add_annotation(text="d) Santa Cruz ({:.1f}%; {:.1f}°S)".format(pmet_metadata.glacier_cover.loc["X00002802"], -pmet_metadata.gauge_lat.loc["X00002802"]),     
                   font=dict(size=16), xref = "paper", yref = "paper", x=0,  y=0, showarrow=False)

fig.add_annotation(text="e) Grey ({:.1f}% ; {:.1f}°S)".format(pmet_metadata.glacier_cover.loc["X12287001"], -pmet_metadata.gauge_lat.loc["X12287001"]),           
                   font=dict(size=16), xref = "paper", yref = "paper", x=0.44,  y=0, showarrow=False)

fig.add_annotation(text="f) Serrano ({:.1f}%; {:.1f}°S)".format(pmet_metadata.glacier_cover.loc["X12289001"], -pmet_metadata.gauge_lat.loc["X12289001"]),        
                   font=dict(size=16), xref = "paper", yref = "paper", x=0.88,  y=0, showarrow=False)

# legends 
fig.add_trace(go.Scatter(x=[None], y=[None],  mode='lines', name= "GR4J + OGGM", line=dict(color= cl[1], width = 1.2, dash = "dot"), showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=[None], y=[None],  mode='lines', name= "TUWmodel + OGGM", line=dict(color= cl[4], width = 1.2, dash = "dot"), showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=[None], y=[None],  mode='lines', name= "Only LSTM", line=dict(color= cl[2], width = 1.2, dash = "dot"), showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=[None], y=[None],  mode='lines', name= "LSTM + OGGM", line=dict(color= cl[0], width = 1.5), showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=[None], y=[None],  mode='lines', name= "Observed", line=dict(color= "black", width = 1.5), showlegend=True), row=1, col=1)


fig.update_yaxes(range = [0,14], row=1, col=1, title= "Runoff (mm d<sup>-1</sup>)",  tickfont = dict(size=14))
fig.update_yaxes(range = [0,8],  row=2, col=1, title= "Runoff (mm d<sup>-1</sup>)",  tickfont = dict(size=14))
fig.update_yaxes(range = [0,6],  row=1, col=2)
fig.update_yaxes(range = [0,35],  row=2, col=2)
fig.update_yaxes(range = [0,9], row=1, col=3)
fig.update_yaxes(range = [0,12],  row=2, col=3)
fig.update_yaxes(ticks="outside", griddash = "dot")

fig.update_xaxes(ticks="outside", griddash = "dot", range = [0, 365],
                     tickvals=[15,46,74,105,135,166,196,227,258,288,319,349], tickfont = dict(size=15),
                     ticktext = ["J", "F", "M", "A", "M", "J", "J", "A", "S", "O", "N", "D"])

fig.update_layout(autosize = False, width = 1100, height = 650, template = "seaborn", margin = dict(l=10, r=10, b=10, t=10))
fig.update_layout(legend=dict(y=1.05, x = 0.36, orientation="h", bgcolor = 'rgba(0,0,0,0.0)', font_size = 13))
fig.update_annotations(font_size=15)

fig.write_image("reports/figures/FigureS2_Seasonal_performance_{}.png".format(exp), scale = 4)
#fig.show()